In [1]:
import os,glob
from skimage import segmentation,feature,morphology,exposure,measure,graph,filters,util
import tifffile
import matplotlib.pyplot as plt
from skimage.util import invert
import numpy as np
import pandas as pd
import timeit
import re
import multiprocessing as mp

In [2]:
cd ~/PycharmProjects/IMYG/venv/lib/python3.7/site-packages

/Users/rubinlab/PycharmProjects/IMYG/venv/lib/python3.7/site-packages


In [3]:
import IMyG as im

In [4]:
def generate_96_well_plate():
    rows = ["A","B","C","D","E","F","G","G"]
    columns = np.linspace(1,12,12).astype(int)
    plate = pd.DataFrame(0,index=rows,columns=columns)
    return(plate)
    
def read_from_plate(input_path,plate):
    files = glob.glob(input_path+"*.nd2")
    output_path = input_path+"output/"
    pattern = re.compile("Well([A-Z])([0-9]{1,2})")
    if len(files) == 0:
        print("Empty folder!")
    else:
        print("%d .nd2 files found in input directory."%(len(files)))
    if not os.path.isdir(output_path):
        os.mkdir(output_path)
    count_plate = generate_96_well_plate()
    for i in range(0,len(files),4):
        processes = []
        for j in range(0,min(4,len(files)-i)):
            file = files[i+j]
            row,column = re.search(pattern,file).group(1),re.search(pattern,file).group(2)
            count = count_plate[int(column)][row]
            count_plate[int(column)][row]+=1
            header = "{}_{}{}_{}".format(plate,row,column,count)
            processes.append(mp.Process(target=im.IO.process_nd2,args=(file,output_path,header)))
        for p in processes:
            p.start()
        for p in processes:
            p.join()
    count_plate.to_excel(output_path+"plate_count.xls")
    return(count_plate)

In [6]:
read_from_plate("/Volumes/Sam/MSR1_IDW/plate1_raw_files/","MSR1")

112 .nd2 files found in input directory.
Error found!
Error found while optimizing skeleton!
Error found!


,1,2,3,4,5,6,7,8,9,10,11,12
A,4,4,4,4,0,0,4,2,0,1,1,3
B,4,0,3,4,3,0,0,0,0,3,2,3
C,4,1,3,4,4,4,3,3,4,4,4,2
D,0,0,0,0,0,0,0,0,0,0,0,3
E,4,3,4,3,2,1,3,0,0,0,0,0
F,0,0,0,0,0,0,0,0,0,0,0,0
G,0,0,0,0,0,0,0,0,0,0,0,0
G,0,0,0,0,0,0,0,0,0,0,0,0


In [6]:
files = glob.glob("/Volumes/Sam/MSR1_IDW/plate1_raw_files/"+"*.nd2")
im.IO.process_nd2(files[0],"/Users/rubinlab/Desktop/","test")

NameError: name 'im' is not defined